# Ratterdam Information Theoretic Analyses
#### WH July 2019

#### Goal is to prototype information theoretic analyses to address, primarily, if there is a texture response at the single unit level.
#### The first analyses will focus on mutual information

In [2]:
#### Imports 
import numpy as np
import utility_fx as util
import os
import matplotlib.gridspec as gridspec
from matplotlib import pyplot as plt
import scipy
import bisect

import ratterdam_CoreDataStructures as Core
import ratterdam_ParseBehavior as Parse
import ratterdam_Defaults as Def
import ratterdam_visBasic as Vis
import ratterdam_DataFiltering as Filt

In [3]:
%qtconsole --style native
%matplotlib qt5

### Mutual Information - Skaggs
#### Using the Skaggs 1993 definition of mutual information. 
##### Considers average firing rate, not trial  by trial. Significance will be computed by shuffling trial labels and recomputing the null averages and then MI
##### Formula: I = Sigma ( lambda(x) * log2(lambda(x)/lambda) * p(x), where lambda(x) is average firing rate under symbol x, lambda is overall firing rate, and p(x) is probability of that symbol

In [3]:
datafile = "E:\\Ratterdam\\R808\\R808_Beltway_D5\\"
expCode = "BRD5"
beltwayAlleys = [16,17,3,1,5,7,8,10,11]
rat = "R808"
alleyTracking, alleyVisits,  txtVisits, p_sess, ts_sess = Parse.getDaysBehavioralData(datafile, expCode)
stimfiles = Parse.getStimFileName(datafile)
stimData = Parse.loadBeltwayData(datafile,stimfiles,expCode)

50


In [784]:
clustname = 'TT15\\cl-maze1.4'
unit = Core.UnitData(clustname, datafile, expCode, Def.alleyBounds, alleyVisits, txtVisits, p_sess, ts_sess)
unit.loadData_raw()

C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:170: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:170: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:176: RuntimeWarning: divide by zero encountered in reciprocal
  n = (ls* np.reciprocal(lo)) * 30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:176: RuntimeWarning: invalid value encountered in multiply
  n = (ls* np.reciprocal(lo)) * 30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\utility_fx.py:324: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW


In [5]:
def skaggsMutualInfo(firingRates, symbolCounts,bins=[0,Def.singleAlleyBins[0]-1]):
    """ Uses the skaggs 1993 definition of mutual information
    to compute MI between fr and stimulus using average fr f.ea. condition
    
    This fx computes MI in a binned range of the alley. Pass [start, stop]
    inclusive to set range within which MI is computed. Default is whole alley
    
    firingRates is a dict of {symbol:avg_FR_vec} structure. For example
    A:[2.1,2,0,0,...,1].
    
    symbolCounts is a dict of {symbol:NumTrials} where numTrials is number of trials
    having that stimulus present. used in p(x) calc
    """
    #calc overall fr average across stimuli
    allData = []
    for _, data in firingRates.items():
        allData.extend(data[bins[0]:bins[1]+1]) #add data within range you want
    grandMean = np.mean(allData)
        
    skaggsScore = 0
    for symbol, data in firingRates.items():
        dataExtent = data[bins[0]:bins[1]+1] # get the range of data you want
        symbolProb = symbolCounts[symbol] / sum([num for num in symbolCounts.values()])  # this is p(x) in skaggs 1993. Prob of the symbol occurnig, just # trials of it
        skaggsScore += I
        
    return skaggsScore

In [6]:
def calcAvgFiringRates(unit, alley, shuffle=False):
    """Create a matrix of firing rates where rows are trials
    and columns are spatial positions binned. These correspond
    to labels of stimulus during that trial. Take average of all
    trials (subset of matrix) with a given label and put into a dict
    of form {label:avgRateVec}. If shuffle == True then the labels 
    are shuffled first to get null avg rate vectors"""
    
    firingRates  = {}
    
    labels = [visit['metadata']['stimulus'] for visit in unit.alleys[alley]]
    if shuffle:
        labels = np.random.permutation(labels) # np.random.permutation returns a new array instead of shuffling in place
    labels = np.asarray(labels)
    trials = np.empty((0, Def.singleAlleyBins[0]-1))
    for visit in unit.alleys[alley]:
        trials = np.vstack((trials, visit['ratemap1d']))
        
    for stimulus in set(labels):
        subsetmat = trials[labels==stimulus]
        mask = np.ma.masked_invalid(subsetmat)
        firingRates[stimulus] = mask.mean(axis=0) # ignores inf and nan
        
    return firingRates

In [7]:
def calcSymbolCounts(unit,alley):
    """For each stimulus A,B,C get counts
    of how many times that stimulus was presented
    and return in a dict of form {stimulus:count}
    Note that in a standard ratterdam session these will
    be the same and equal to 10 or 20. But sometimes a couple extra
    trials are added for whatever reason"""
    
    symbolCounts = {}
    labels = [visit['metadata']['stimulus'] for visit in unit.alleys[alley]]
    
    for stimulus in set(labels):
        symbolCounts[stimulus] = labels.count(stimulus)
        
    return symbolCounts

In [116]:
alley=16
n=1000
symbolCounts = calcSymbolCounts(unit,alley)

firingRates = calcAvgFiringRates(unit, alley, shuffle=False)
realSkaggsMI = skaggsMutualInfo(firingRates, symbolCounts,bins=[3,7])

nullSkaggsMIs = np.empty((0))
for i in range(n):
    frshuff = calcAvgFiringRates(unit,alley,shuffle=True)
    nullscore = skaggsMutualInfo(frshuff, symbolCounts, bins=[3,7])
    nullSkaggsMIs = np.hstack((nullSkaggsMIs, nullscore))

In [117]:
plt.hist(nullSkaggsMIs,bins=25)
plt.vlines(np.percentile(nullSkaggsMIs,95),-1,100,'k')
plt.vlines(realSkaggsMI,-1,100,'r')

### Standard Mutual Information
#### Traditional definition of MI using joint,conditional, and marginal probabilities
#### I(X|Y) = Sigma_i,j( p(yi|xj) * log2(p(yi|xj)/p(yi)) * p(xj))
#### Uses contingency tables of spikes (or FR) emitted on each trial under each stimulus.

In [112]:
def makeSymbolBins(alley):
    mat  = np.empty((0, Def.singleAlleyBins[0]-1))
    for i,visit in enumerate(unit.alleys[alley]):
        mat = np.vstack((mat, visit['ratemap1d']))
    flatmat = np.ndarray.flatten(mat)
    mean = np.nanmean(flatmat)
    std = np.nanstd(flatmat)
    
    freedman_diaconis_binWidth = 2*(scipy.stats.iqr(flatmat, nan_policy='omit')/np.cbrt(flatmat.shape[0]))
    numbins = round(np.nanmax(flatmat)/freedman_diaconis_binWidth)
    symbolBins = np.linspace(0, np.nanmax(flatmat), int(numbins)) 
    
    return symbolBins

In [113]:
def calcSymbol(data, symbolBins):
    return bisect.bisect_left(symbolBins, np.nanmax(data))

In [80]:
def calcMutualInformation_old(alley, shuffle):
    """
    Calculate MI using I(X|Y) = Sigma_i,j( p(yi|xj) * log2(p(yi|xj)/p(yi)) * p(xj))
    """
    trial_rates = []
    trial_symbols = []
    symbol_rates = {'A':[], 'B':[], 'C':[]}
    symbolBins = makeSymbolBins(alley)

    trial_symbol_list = [visit['metadata']['stimulus'] for visit in unit.alleys[alley]]

    if shuffle == True:
        trial_symbol_list = np.random.permutation(trial_symbol_list)

    for i,visit in enumerate(unit.alleys[alley]):
        txtsymbol = trial_symbol_list[i]
        ratemap = visit['ratemap1d']
        mask = np.ma.masked_invalid(ratemap)
        frSymbol = calcSymbol(mask, symbolBins)
        trial_rates.append(frSymbol)
        trial_symbols.append(txtsymbol)
        symbol_rates[txtsymbol].append(frSymbol)
        trial_rates = sorted(trial_rates)

    symbol_rate_counts = {'A':[],'B':[], 'C':[]}
    for symbol in ['A','B','C']:
        for observedRate in set(trial_rates):
            symbol_rate_counts[symbol].append(symbol_rates[symbol].count(observedRate))

    I = 0
    for symbol in ['A', 'B', 'C']:
        for i in range(len(symbolBins)):
            px = trial_symbols.count(symbol) / len(trial_symbols)
            py = trial_rates.count(i)/len(trial_rates)
            pxy = symbol_rate_counts[symbol][i] / len(symbolBins)
            if pxy == 0 or py==0:
                pseudoI = 0
            else:
                pseudoI = pxy * np.log2(pxy/(px*py))
            I += pseudoI
    return I

In [116]:
alley = 1
realI = calcMutualInformation_old(alley, False)
nulls = []
n=1000
thresh = round((0.05/9),3)*n
for i in range(n):
    nullI = calcMutualInformation(alley,True)
    nulls.append(nullI)
nulls = sorted(nulls)
plt.figure()
plt.hist(nulls)
plt.vlines(realI,0,200,'r')
plt.vlines(nulls[int(-1*thresh)],0,100,'k')
plt.title(f"{unit.name} Alley {beltwayAlleyLookup[alley]}, whole alley MI")

IndexError: list index out of range

In [11]:
# get sliding window MI values
alley = 1
p90s = []
p10s = []
realIs = []
allbins = [[0,4],[1,5],[2,6],[3,7],[4,8],[5,9],[6,10],[7,11],[8,12],[9,12],[10,12]]
for b in allbins:
    bins = [b[0], b[1]]
    realI = calcMutualInformation(alley, bins, False)
    realIs.append(realI)
    nulls = []

    for i in range(1000):
        nullI = calcMutualInformation(alley, bins,True)
        nulls.append(nullI)
    pup = np.percentile(nulls,99.75)
    plow = np.percentile(nulls,0.25)
    pupss.append(p90)
    p10s.append(p10)
plt.figure()
plt.plot(realIs,'g')
plt.fill_between(range(len(p10s)),p10s,p90s,alpha=0.5)
plt.title(f"{expCode} {unit.name} Alley {alley}, windowed MI")
plt.xticks(range(len(allbins)), allbins)

([<matplotlib.axis.XTick at 0x229761d69e8>,
 <a list of 11 Text xticklabel objects>)

In [210]:
### Subplots of all alleys for a cell - whole track

datafile = "E:\\Ratterdam\\R808\\R808_Beltway_D4\\"
rat = 'R808'
expCode = "BRD4"
figurePath = 'E:\\Ratterdam\\multidayFigures\\randomForest\\'
alleyTracking, alleyVisits,  txtVisits, p_sess, ts_sess = Parse.getDaysBehavioralData(datafile, expCode)
population = {}
for subdir, dirs, fs in os.walk(datafile):
    for f in fs:
        if 'cl-maze1' in f and 'OLD' not in f and 'Undefined' not in f:
            clustname = subdir[subdir.index("TT"):] + "\\" + f
            unit = Core.UnitData(clustname, datafile, expCode, Def.alleyBounds, alleyVisits, txtVisits, p_sess, ts_sess)
            unit.loadData_raw()



n=1000
thresh = round((0.05/9),3)*n
fig, ax = plt.subplots(3,3)
for i, alley in enumerate(Def.beltwayAlleys):
    axis = fig.axes[i]
    realI = calcMutualInformation(alley, False)
    nulls = []
    
    for i in range(n):
        nullI = calcMutualInformation(alley,True)
        nulls.append(nullI)
    axis.hist(nulls)
    axis.vlines(realI,0,200,'r')
    nulls = sorted(nulls)
    #axis.vlines(np.percentile(nulls,99.75),0,100,'k')
    axis.vlines(nulls[int(-1*thresh)],0,100,'k')
    axis.set_title(f"Alley {Def.beltwayAlleyLookup[alley]}")
plt.suptitle(f"Whole Alley MI Unit {expCode} {unit.name}")

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


OverflowError: cannot convert float infinity to integer

In [22]:
beltwayAlleyLookup = {16:1, 17:2, 3:3, 1:4, 5:5, 7:6, 8:7, 10:8, 11:9}

In [336]:
alley = 11
shuffle = False

In [73]:
def calcMIV2(alley, shuffle):
    # bitstring based MI fx
    visits = np.empty((0, Def.singleAlleyBins[0]-1))
    stimSymbols = []
    binnedVisits = {'A':None, 'B':None, 'C':None}
    trial_symbol_list = [visit['metadata']['stimulus'] for visit in unit.alleys[alley]]
    if shuffle == True:
        trial_symbol_list = np.random.permutation(trial_symbol_list)

    for i,visit in enumerate(unit.alleys[alley]):
        visits = np.vstack((visits, visit['ratemap1d']))
        stimSymbols.append(trial_symbol_list[i]) # do like this for indirect shuffling when needed

    stimSymbols = np.asarray(stimSymbols)
    means = np.mean(visits, axis=0)
    for s in ['A','B','C']:
        binnedVisits[s] = (visits[np.where(stimSymbols==s)]>means)*1 #hack to convert bools to ints. 

    allBinnedVisits = (visits>means)*1

    frSymbolSet = list(itertools.product([0,1],repeat=Def.singleAlleyBins[0]-1))

    symbol_rate_counts = {'A':[],'B':[], 'C':[]}
    for s in ['A','B','C']:
        for frSymbol in frSymbolSet:
            c = np.where((binnedVisits[s] == frSymbol).all(axis=1))[0].shape[0]
            symbol_rate_counts[s].append(c)

    I = 0
    for s in ['A', 'B', 'C']:
        for i, frSymbol in enumerate(frSymbolSet):
            px = np.where(stimSymbols==s)[0].shape[0] / len(stimSymbols)
            py = np.where((allBinnedVisits == frSymbol).all(axis=1))[0].shape[0] / allBinnedVisits.shape[0]
            try:
                pxy = symbol_rate_counts[s][i] / sum(symbol_rate_counts['A'] + symbol_rate_counts['B'] + symbol_rate_counts['C'])
            except ZeroDivisionError:
                pxy = 0
            if pxy == 0:
                pseudoI = 0
            else:
                pseudoI = pxy * np.log2(pxy/(px*py))
            I += pseudoI
    return I

In [440]:
realI = calcMIV2(11,False)

In [442]:
for i in range(1000):
    n = calcMIV2(11,True)
    nulls.append(n)

## Redoing MI in a more readable, matrix based way. All those nested dicts confused me 

In [241]:
rat = "R781"
expCode = "BRD3"
clustname = 'TT11\\cl-maze1.1'
datafile = f'E:\\Ratterdam\\{rat}\\{rat}{expCode}\\'
figurePath = f'E:\\Ratterdam\\{rat}\\mutual_information\\{expCode}\\'
alleyTracking, alleyVisits,  txtVisits, p_sess, ts_sess = Parse.getDaysBehavioralData(datafile, expCode)
unit = Core.UnitData(clustname, datafile, expCode, Def.alleyBounds, alleyVisits, txtVisits, p_sess, ts_sess)
unit.loadData_raw()

C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:172: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:172: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:178: RuntimeWarning: divide by zero encountered in reciprocal
  n = (ls* np.reciprocal(lo)) * 30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:178: RuntimeWarning: invalid value encountered in multiply
  n = (ls* np.reciprocal(lo)) * 30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\utility_fx.py:329: RuntimeWarning: invalid value encountered in true_divide
  def getClustList(datafile):


In [140]:
def createNeuralSymbols():
    '''
    Hardcoded based on neural fr alphabet you want. All possible bins or bitstrings
    depending on what kind of alphabet.
    '''
    #symbolBins = list(itertools.product(list(range(1,51)),repeat=Def.singleAlleyBins[0]-1))
    mat  = np.empty((0, Def.singleAlleyBins[0]-1))
    for i,visit in enumerate(unit.alleys[alley]):
        mat = np.vstack((mat, visit['ratemap1d']))
    flatmat = np.ndarray.flatten(mat)
    mean = np.nanmean(flatmat)
    std = np.nanstd(flatmat)
    
    freedman_diaconis_binWidth = 2*(scipy.stats.iqr(flatmat, nan_policy='omit')/np.cbrt(flatmat.shape[0]))
    numbins = round(np.nanmax(flatmat)/freedman_diaconis_binWidth)
    try:
        symbolBins = np.linspace(0, np.nanmax(flatmat), int(numbins)) 
    except:
        symbolBins = np.linspace(0, np.nanmax(flatmat), 40)
    return symbolBins

In [141]:
def convertActivitytoSymbols(unit, alley):
#     mat  = np.empty((0, Def.singleAlleyBins[0]-1))
#     for i,visit in enumerate(unit.alleys[1]):
#         mat = np.vstack((mat, visit['ratemap1d']))
#     flatmat = np.ndarray.flatten(mat)
#     mean = np.nanmean(mat,axis=0)
#     print(mean.shape)
#     return (mat>mean)*1 #converts to bool arr in parens, then converts to int. e.g. true*1 = 1.
    vec  = np.empty((0,1))
    for i,visit in enumerate(unit.alleys[alley]):
        c = calcSymbol(visit['ratemap1d'], neural_symbols_set)
        vec = np.vstack((vec, c))
    return vec

In [146]:
def calcSymbol(data, symbolBins):
    return bisect.bisect_left(symbolBins, np.nanmax(data))

In [157]:
def calculateMutualInformation(neural_symbols_set, converted_array, shuffle):

    trialStimSymbols = np.asarray([visit['metadata']['stimulus'] for visit in unit.alleys[alley]])
    if shuffle == True:
        trialStimSymbols = np.random.permutation(trialStimSymbols)
    joint_table = np.zeros((len(neural_symbols_set),0))
    for i,stim in enumerate(['A','B','C']):
        p_txty = np.zeros((0,1))
        for j,sym in enumerate(neural_symbols_set):
            c = np.intersect1d(np.where((converted_array == sym).all(axis=1))[0],np.where(trialStimSymbols==stim)[0]).shape[0]
            p_txty = np.vstack((p_txty,c))
        joint_table  = np.hstack((joint_table, p_txty))

    mi = 0
    for i in range(3):
        for j in range(len(neural_symbols_set)):
            px = joint_table.sum(axis=0)[i]/joint_table.sum()
            py = joint_table.sum(axis=1)[j]/joint_table.sum()
            pxy = joint_table[j,i]/joint_table.sum()
            if pxy != 0:
                mi += pxy * np.log2(pxy/(px*py))
    return mi

In [98]:
def calcMutualInformation_old(alley, shuffle):
    """
    Calculate MI using I(X|Y) = Sigma_i,j( p(yi|xj) * log2(p(yi|xj)/p(yi)) * p(xj))
    """
    trial_rates = []
    trial_symbols = []
    symbol_rates = {'A':[], 'B':[], 'C':[]}
#     symbolBins = makeSymbolBins(alley)
    symbolBins = createNeuralSymbols()

    trial_symbol_list = [visit['metadata']['stimulus'] for visit in unit.alleys[alley]]

    if shuffle == True:
        trial_symbol_list = np.random.permutation(trial_symbol_list)

    for i,visit in enumerate(unit.alleys[alley]):
        txtsymbol = trial_symbol_list[i]
        ratemap = visit['ratemap1d']
        mask = np.ma.masked_invalid(ratemap)
        frSymbol = calcSymbol(mask, symbolBins)
        trial_rates.append(frSymbol)
        trial_symbols.append(txtsymbol)
        symbol_rates[txtsymbol].append(frSymbol)
        trial_rates = sorted(trial_rates)

    symbol_rate_counts = {'A':[],'B':[], 'C':[]}
    for symbol in ['A','B','C']:
        for observedRate in set(trial_rates):
            symbol_rate_counts[symbol].append(symbol_rates[symbol].count(observedRate))

    I = 0
    for symbol in ['A', 'B', 'C']:
        for i in range(len(symbolBins)):
            px = trial_symbols.count(symbol) / len(trial_symbols)
            py = trial_rates.count(trial_rates[i])/len(trial_rates)
            print(i)
            print(len(symbol_rate_counts[symbol]))
            pxy = symbol_rate_counts[symbol][i] / len(symbolBins)
            if pxy == 0 or py==0:
                pseudoI = 0
            else:
                pseudoI = pxy * np.log2(pxy/(px*py))
            I += pseudoI
    return I

In [81]:
### Subplots of all alleys for a cell - whole track
rat = "R781"
expCode = "BRD3"
datafile = f'E:\\Ratterdam\\{rat}\\{rat}{expCode}\\'
figurePath = f'E:\\Ratterdam\\{rat}\\mutual_information\\{expCode}\\'
alleyTracking, alleyVisits,  txtVisits, p_sess, ts_sess = Parse.getDaysBehavioralData(datafile, expCode)
population = {}
allRealIs = {i:[] for i in Def.beltwayAlleys}
stamp = util.genTimestamp()
n=1000 # num shuffles
for subdir, dirs, fs in os.walk(datafile):
    for f in fs:
        if 'cl-maze1' in f and 'OLD' not in f and 'Undefined' not in f:
            clustname = subdir[subdir.index("TT"):] + "\\" + f
            unit = Core.UnitData(clustname, datafile, expCode, Def.alleyBounds, alleyVisits, txtVisits, p_sess, ts_sess)
            unit.loadData_raw()
            rm = util.makeRM(unit.spikes, unit.position)            
            if np.nanpercentile(rm,Def.wholetrack_imshow_pct_cutoff) >= 1.:
                print(clustname)


                # Check how many alleys have some appreciable activity and adjust your Bonf. correction by that
                validalleys = []
                for a in Def.beltwayAlleys:
                    valid = Filt.checkMinimumPassesActivity(unit, a)
                    validalleys.append(valid)
                multCompFactor = sum(validalleys) # we will adjust for multiple comparisons by considering the # alleys where there is 
                                                # appreciable activity (avg 1Hz overall) and therefore a valid test to even attempt
                if multCompFactor >= 1:
                    thresh = round((0.05/multCompFactor),2)*n


#                     fig, ax = plt.subplots(3,3,figsize=(8,8))
                    for i, alley in enumerate(Def.beltwayAlleys):
#                         axis = fig.axes[i]
                        if  validalleys[i] == True:
                            stimulus_symbols = ['A','B','C']
                            neural_symbols_set = createNeuralSymbols()
                            converted_array = convertActivitytoSymbols(unit, alley)
#                             neural_symbols_set = range(len(neural_symbols_set)) 
#                             realI = calculateMutualInformation(neural_symbols_set, converted_array, False)
                            realI = calcMutualInformation_old(alley, shuffle=False)
                            print(f"{Def.beltwayAlleyLookup[alley]} MI: {realI}")
                            allRealIs[alley].append(realI)

#                             nulls = []
#                             for i in range(n):
#                                 nullI = calculateMutualInformation(neural_symbols_set, converted_array, True)
#                                 nulls.append(nullI)
#                             axis.hist(nulls,50)
#                             axis.vlines(realI,0,150,'r')
#                             nulls = sorted(nulls)
#                             axis.vlines(nulls[int(-1*thresh)],0,150,'grey')
#                             axis.set_title(f"Alley {Def.beltwayAlleyLookup[alley]}, {round(scipy.stats.percentileofscore(nulls,realI),2)}")
                        else:
#                             axis.set_title(f"Insufficient Activity Alley {Def.beltwayAlleyLookup[alley]}")
                            print(f"{Def.beltwayAlleyLookup[alley]} not included in MI calc")

#                     plt.suptitle(f"Whole Alley MI Unit {expCode} {unit.name}")

#                     plt.savefig(figurePath+f"{stamp}_{unit.name}_alleyMI.png", format='png',dpi=300)
#                     plt.close()

C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:172: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:172: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:178: RuntimeWarning: divide by zero encountered in reciprocal
  n = (ls* np.reciprocal(lo)) * 30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:178: RuntimeWarning: invalid value encountered in multiply
  n = (ls* np.reciprocal(lo)) * 30


TT11\cl-maze1.1


C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\utility_fx.py:329: RuntimeWarning: invalid value encountered in true_divide
  def getClustList(datafile):
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\utility_fx.py:502: RuntimeWarning: divide by zero encountered in reciprocal
  n[np.where(ho==0)] = np.nan
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\utility_fx.py:502: RuntimeWarning: invalid value encountered in multiply
  n[np.where(ho==0)] = np.nan


IndexError: list index out of range

In [981]:
neural_symbols_set = createNeuralSymbols()
mm = np.nanmax(mat[:,0])
a = np.digitize(mat[:,0], bins=[])

mm = np.nanmax(mat[:,1])
b = np.digitize(mat[:,1], bins=[])

mm = np.nanmax(mat[:,2])
c = np.digitize(mat[:,2], bins=[])
converted_array = np.column_stack((a,b,c))

In [126]:
mi = calculateMutualInformation(neural_symbols_set, converted_array, False)
nulls = []
for i in range(1000):
    n = calculateMutualInformation(neural_symbols_set, converted_array, True)
    nulls.append(n)
nulls = sorted(nulls)
thresh = (0.05/3)*1000
plt.hist(nulls,50)
plt.vlines(mi,0,100,'r')
plt.vlines(np.percentile(nulls[-1*int(thresh)],95),0, 100,'k')

In [242]:
validalleys = []
for a in Def.beltwayAlleys:
    valid = Filt.checkMinimumPassesActivity(unit, a)
    validalleys.append(valid)

for i, alley in enumerate(Def.beltwayAlleys):
    print(f"{Def.beltwayAlleyLookup[alley]}")
    if True:
        stimulus_symbols = ['A','B','C']
        neural_symbols_set = createNeuralSymbols()
        converted_array = convertActivitytoSymbols(unit, alley)
        neural_symbols_set = range(len(neural_symbols_set)) # this converts array of bin edges to array of bin idx
        realI = calculateMutualInformation(neural_symbols_set, converted_array, False)
#       realI = calcMutualInformation_old(alley, shuffle=False)
        print(realI)
    else:
        print(f"{Def.beltwayAlleyLookup[alley]} - insufficient activity")

1
0.5633254510409311
2
0.9779586942468093
3
0.7264661592006296
4
0.7011207838442461
5
0.5634425677041557
6
0.8997345018097208
7
0.8176816642737454
8
0.9299434144670902
9
0.9238233682539165


In [243]:
%qtconsole --style native